In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv


# Fixstars Amplify SDK
import amplify
from amplify import (
    FixstarsClient, 
    VariableGenerator, 
    Model, 
    solve, 
    ToshibaSQBM2Client
)

# System tools
import time
import datetime

# Fixstars AE credentials (if local)
key = os.getenv("SQBM_KEY")
client = ToshibaSQBM2Client()
client.token = key
client.solver = "Qubo"

## Toshiba has a maximum of ten seconds per job on free plan
client.parameters.timeout = 1 # 1 second

# Print SDK version
print(f"Amplify SDK version: {amplify.__version__}")

Amplify SDK version: 1.3.1


In [5]:
# Tiny linear regression example
X_data = np.array([1, 2, 3])
y_data = np.array([2, 3, 5])

# Design matrix: add bias column
X = np.vstack([np.ones_like(X_data), X_data]).T

print("Design matrix X:\n", X)
print("Target y:\n", y_data)

Design matrix X:
 [[1 1]
 [1 2]
 [1 3]]
Target y:
 [2 3 5]


In [6]:
# Normal equations
A = X.T @ X
b = X.T @ y_data

print("Normal equations:")
print("A:\n", A)
print("b:", b)

# Check SPD
print("Is A symmetric?", np.allclose(A, A.T))
print("Eigenvalues:", np.linalg.eigvalsh(A))


Normal equations:
A:
 [[ 3  6]
 [ 6 14]]
b: [10 23]
Is A symmetric? True
Eigenvalues: [ 0.3605897 16.6394103]


In [7]:
# Box Algorithm parameters
d = A.shape[0]
c = np.zeros(d)
L = 1.0
beta = 0.2
epsilon = 1e-6
max_iterations = 50

best_energy = np.inf
num_translations = 0
num_contractions = 0

# For stats
total_wall_time = 0.0
total_solver_time = 0.0

print(f"Box Algorithm starting: dimension={d}, beta={beta}, epsilon={epsilon}")

Box Algorithm starting: dimension=2, beta=0.2, epsilon=1e-06


In [8]:
L_history = []
energy_history = []
wall_time_history = []

for iteration in range(1, max_iterations + 1):
    gen = VariableGenerator()
    q1 = gen.array("Binary", d)
    q2 = gen.array("Binary", d)
    
    # 2) Express w in terms of q1, q2
    w = c + L * (-2 * q1 + q2)
    
    # 3) Build energy
    energy = 0.5 * (w @ (A @ w)) - b @ w
    
    # 4) Build model
    model = Model(energy)
    
    # 5) Solve and time
    wall_start = time.time()
    result = solve(model, client)
    wall_end = time.time()
    
    wall_time = wall_end - wall_start
    response_time = result.response_time.total_seconds()
    execution_time = result.execution_time.total_seconds()
    
    total_wall_time += wall_time
    total_solver_time += execution_time
    
    if len(result) == 0:
        print(f"Iteration {iteration}: No feasible solution, stopping.")
        break
    
    solution = result.best
    q1_sol = q1.evaluate(solution.values)
    q2_sol = q2.evaluate(solution.values)
    
    w_new = c + L * (-2 * q1_sol + q2_sol)
    energy_val = solution.objective

    improved = energy_val < best_energy
    
    print(f"Iter {iteration:2d} | E: {energy_val:.6f} | {'translate' if improved else 'contract'} | "
          f"L: {L:.3e} | wall: {wall_time:.2f}s | solver: {execution_time:.2f}s")
    
    if improved:
        c = w_new  # translate box
        best_energy = energy_val
        num_translations += 1
    else:
        L = beta * L  # contract box
        num_contractions += 1
    
    
    L_history.append(L)
    energy_history.append(energy_val)
    wall_time_history.append(wall_time)
    
    # Stop if L tiny
    if L < epsilon:
        print("Converged: L below epsilon.")
        break

print("\nFinal estimated w:", c)
print("Direct solve w:", np.linalg.solve(A, b))
print(f"Total translations: {num_translations}, contractions: {num_contractions}")
print(f"Total wall time: {total_wall_time:.2f}s")
print(f"Total solver time: {total_solver_time:.2f}s")

RuntimeError: 400: Bad Request, timeout too large


In [ ]:
import matplotlib.pyplot as plt

if "L_history" not in locals():
    print("No iteration history found! Please store L, energy, and wall time inside the loop next run.")
else:
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))

    # --- Plot L ---
    axes[0].plot(L_history, marker='o')
    axes[0].set_title("Box Size L vs Iteration")
    axes[0].set_xlabel("Iteration")
    axes[0].set_ylabel("L")
    axes[0].set_yscale("log")
    axes[0].grid(True)

    # --- Plot energy ---
    axes[1].plot(energy_history, marker='o')
    axes[1].set_title("Objective (Energy) vs Iteration")
    axes[1].set_xlabel("Iteration")
    axes[1].set_ylabel("Energy")
    axes[1].grid(True)

    # --- Plot wall time ---
    axes[2].plot(wall_time_history, marker='o')
    axes[2].set_title("Wall Time per Solve")
    axes[2].set_xlabel("Iteration")
    axes[2].set_ylabel("Seconds")
    axes[2].grid(True)

    plt.tight_layout()
    plt.show()